![](https://drive.google.com/uc?export=view&id=1-5X9OUkA-C2Ih1gOS9Jd7GmkTWUEpDg1)

# Laboratorio 03: Procesamiento de lenguaje natural con Python
## Introducción a _Data Science_

<!--<center>
    <img src='images/GAN.jpeg'style="width: 600px;">
</center>-->

**Profesor**: Juan Bekios Calfa

**Carreras**: ICCI, ITI, ICI

<!--<sub><sup>Tutorial: GANS. Sensio Artificial Intelligence [link](https://sensioai.com/blog/051_gans)</sup></sub> -->



---
**Nombre del alumno**: --Indicar nombre completo--

**Carrera**: -- Indicar carrera --

---

# 1. Introducción

El laboratorio se centra en el tratamiento de **datos de texto libre**.  Además, de los **datos estructurado**s y de los **grafos** que hemos analizado anteriormente, el **texto libre** constituye uno de los tipos más comunes de datos "ampliamente disponibles": 

* Las páginas web
* Los campos de "comentarios" no estructurados de muchas bases de datos relacionales 
* Otras fuentes de datos de gran tamaño fáciles de obtener vienen naturalmente en forma de texto libre.  

La diferencia notable, por supuesto, es que, a diferencia de los tipos de datos que hemos analizado antes, el texto libre carece de la estructura "fácilmente extraíble" inherente a los tipos de datos anteriores que hemos considerado.

El laboratorio tiene como objetivo conocer algunas técnicas para pre-procesar texto para ser utilizado como entrada a un clasificador o regresor.

Se estudiarán diferentes librerías que permiten preparar el texto para luego codificarlo en una estructura de datos manejable.

## 1.5 Visualización de palabras (TF)

El modelo de bolsa de palabras (*Bag of words*) es, con mucho, el medio más común de representar documentos en la **ciencia de datos**. Según este modelo, un documento se describe únicamente por el conjunto de palabras (y posiblemente su número) que lo componen. Se ignora toda la información sobre el orden real de las palabras. Se trata esencialmente de la llamada "nube de palabras" de un documento.

In [ ]:
from bs4 import BeautifulSoup
import requests
import re

response = requests.get("http://www.datasciencecourse.org")
root = BeautifulSoup(response.content, "lxml")

from wordcloud import WordCloud
wc = WordCloud(width=800,height=400).generate(re.sub(r"\s+"," ", root.text))
wc.to_image()

In [ ]:
!pip install wikipedia

In [ ]:
import wikipedia

wikipedia.set_lang("es")
results_list = wikipedia.search("Barack")

for each_result in results_list:
  print(each_result)

Extraemos dos textos de la base de datos

In [ ]:
bo01 = wikipedia.page("Barack Obama")
bo01.content

In [ ]:
bo02 = wikipedia.page("Moisés Barack")
bo02.content

In [ ]:
wc = WordCloud(width=800,height=400).generate(re.sub(r"\s+"," ", bo01.content))
wc.to_image()

# NLTK

## Instalar librerías

In [ ]:
import nltk
nltk.download('popular')

In [ ]:
# Obtener TOKENs del texto
tokens = nltk.word_tokenize(bo01.content)
print(tokens)

## Algunos comandos

### Eliminar signos de puntuación en el texto

In [ ]:
# Eliminar espacios 
nueva_linea = re.sub(r"\s+"," ", bo01.content)
tokens = nltk.word_tokenize(nueva_linea)
print(tokens)

In [ ]:
from nltk.corpus import stopwords

# Palabras innecesarias
sw = stopwords.words('spanish')

filtered_sentence = []
for w in tokens: 
    if w not in sw: 
        filtered_sentence.append(w)

print(filtered_sentence)

In [ ]:
wc = WordCloud(width=800,height=400).generate(" ".join(filtered_sentence))
wc.to_image()

### Configurar el corpus a español y crearlo

## Obtener los vectores de frecuencia (TF, *term frequency matrix*) 

---

En este caso, podemos representar los documentos mediante una matriz de frecuencias de términos de $m \times n$, donde $m$ indica el número de documentos y $n$ el tamaño del vocabulario (es decir, el número de palabras únicas en todos los documentos).  Para ver (de forma ingenua) cómo construir esta lista, consideremos primero una forma sencilla de obtener una lista de todas las palabras únicas en todos los documentos.  En general, no es necesario ordenar la lista de palabras, pero lo haremos por simplicidad.  Es una buena idea generar también un diccionario que asigne las palabras a su índice en esta lista, ya que con frecuencia querremos buscar el índice correspondiente a una palabra.

In [ ]:
vocab_dict = {k:i for i,k in enumerate(filtered_sentence)}
print(filtered_sentence, "\n")
print(vocab_dict, "\n")

## Vectores de frecuencia utilizando sklearn

Listaremos las palabras que no tienen relevancia en un texto y es preferible eliminarlas (*stop words*).

In [ ]:
# Librerías sklearn - Machine Learning
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

sw = stopwords.words('spanish')
print(sw)

Calculamos el vector TF.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

sw = stopwords.words('spanish')
# print(sw)
vectorizer = CountVectorizer(stop_words=sw)

freq_matrix = vectorizer.fit_transform([bo01.content, bo02.content])
#print(freq_matrix)

feature_names = vectorizer.get_feature_names()
dense_frec = freq_matrix.todense()
denselist_frec = dense_frec.tolist()
df = pd.DataFrame(denselist_frec, columns=feature_names)
df.head()

Calculamos el vector TF eliminando las palabras que contengan caracteres que no sean alfanuméricos.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

sw = stopwords.words('spanish')
# print(sw)
vectorizer = CountVectorizer(stop_words=sw, token_pattern=r'[^\d\W]+')

freq_matrix = vectorizer.fit_transform([bo01.content, bo02.content])
# print(freq_matrix)
feature_names = vectorizer.get_feature_names()

dense_frec = freq_matrix.todense()
denselist_frec = dense_frec.tolist()
df = pd.DataFrame(denselist_frec, columns=feature_names)
df.head()

## Obtener frecuencia inversa entre documentos (idf)

### Frecuencia inversa del documento 

Un problema obvio con el uso de recuentos de frecuencia de términos normales para representar un documento es que el vector del documento (y las similitudes resultantes que consideraremos) a menudo estará "dominado" por palabras muy comunes, por ejemplo "de", "el", "es", en los documentos de ejemplo anteriores.  Este problema puede mitigarse hasta cierto punto excluyendo las llamadas "stop words" (palabras comunes en inglés como "the", "a", "of" que no se consideran relevantes para los documentos concretos) de la matriz de frecuencia de términos.  Pero esto sigue ignorando el caso en el que una palabra que puede no ser una palabra de parada genérica sigue apareciendo en un gran número de documentos.  Intuitivamente, esperamos que las palabras más "importantes" de un documento sean precisamente las que sólo aparecen en un número relativamente pequeño de documentos, por lo que queremos descontar el peso de los términos que aparecen con mucha frecuencia.

Esto puede lograrse mediante el peso de la frecuencia inversa de los documentos para las palabras.  Al igual que con las frecuencias de términos, existen diferentes ponderaciones de este término, pero la formulación más común es

\begin{equation}
\mathrm{idf}_j = \log\left(\frac{\mbox{# documents}}{\mbox{# documents with word $j$}}\right).
\end{equation}

Por ejemplo, si la palabra aparece en todos los documentos, el peso de la frecuencia inversa del documento será cero (logaritmo de uno).  Por el contrario, si una palabra sólo aparece en un documento, su frecuencia documental inversa será $\log (\mbox{# documentos})$.

Tenga en cuenta que la frecuencia inversa de documentos es un término _por palabra_, a diferencia de la frecuencia de términos, que es _por palabra y documento_.  Podemos calcular la frecuencia inversa de documentos para nuestro conjunto de datos de la siguiente manera, que principalmente sólo requiere contar cuántos documentos contienen cada palabra.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

transformer = TfidfTransformer()
tfidf_matrix = transformer.fit_transform(freq_matrix)

dense = tfidf_matrix.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
df.head()

## Obtener frecuencia inversa entre documentos (idf) de forma directa

In [ ]:
vectorizer = TfidfVectorizer(stop_words=sw, token_pattern=r'[^\d\W]+')
vectors = vectorizer.fit_transform([bo01.content, bo02.content])
# print(vectors)
feature_names = vectorizer.get_feature_names()
# print(feature_names)
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
df.head()

# Tarea

1. Obtener los 100 primeros documentos de Wikipedia y crear un corpus.
2. Obtener 5 documentos no utilizados para entrenar el corpus y generar:

    1. Vectores tf.
    2. Vectores idf a partir del vector tf.
    3. Vector idf de manera directa.
    4. **Explicar los resultados.**. ¿Qué problemas observó con las difentes técnicas utilizadas?
    5. Comparar la semejanza de todos los documentos utilizando TF. ¿Cual es el más parecido?. 
    6. Comparar la semejanza de todos los documentos utilizando TF-IDF. ¿Cual es el más parecido?.
    7. A su juicio, ¿cual de las dos codificaciones de texto es mejor?. **Explicar los resultados.**
    
    Calcule la matriz de semejanza para una mejor explicación. Utilice el siguiente ejemplo:
        

|           	| $Documento_3$ 	|  $Documento_4$	 | $Documento_5$ | $Documento_6$| $Documento_7$ 	 |
|-----------	|:-------------:|:------------:|:-------------:|:-------------:|:------------:|
| $Documento_3$ |    1.00  	  |    0.00  	 |    0.00  	  |    0.00  	 |    0.00  	  |
| $Documento_4$ |    0.00  	  |    1.00  	 |    0.00  	  |    0.00  	 |    0.00  	  |
| $Documento_5$ |    0.00  	  |    0.00  	 |    1.00  	  |    0.00  	 |    0.00  	  |
| $Documento_6$ |    0.00  	  |    0.00  	 |    0.00  	  |    1.00  	 |    0.00  	  |
| $Documento_7$ |    0.00  	  |    0.00  	 |    0.00  	  |    0.00  	 |    1.00  	  |

**Fecha de entrega**: Jueves 6 de mayo.

# I. Anexo: Similitud del coseno

Dada una matriz TF-IDF (o simplemente frecuencia de término), una de las preguntas más comunes a abordar es calcular la similitud entre varios documentos en el _corpus_.  Una medida común (métrica) para hacerlo es calcular la similitud del coseno entre dos documentos diferentes. Esto es simplemente un producto interno normalizado entre los vectores que describen cada documento. Específicamente,

\begin{equation}
\mbox{SimilitudCoseno}(x,y) = \frac{x^T y}{\|x\|_2 \cdot \|y\|_2}.
\end{equation}

La **similitud del coseno** es un número entre cero (lo que significa que los dos documentos no comparten términos en común) y uno (lo que significa que los dos documentos tienen exactamente la misma frecuencia de términos o representación TFIDF). De hecho, la similitud del coseno es exactamente la inversa de la distancia Eucliean al cuadrado entre los vectores de documentos normalizados; formalmente, para $\tilde{x} = x / \|x\|_2$ and $\tilde{y} = y / \|y\|_2$,

\begin{equation}
\begin{split}
\frac{1}{2}\|\tilde{x} - \tilde{y}\|_2^2 & = \frac{1}{2}(\tilde{x} - \tilde{y})^T (\tilde{x} - \tilde{y}) \\
& = \frac{1}{2} (\tilde{x}^T \tilde{x} - 2 \tilde{x}^T \tilde{y} + \tilde{y}^T \tilde{y}) \\
& = \frac{1}{2} (1 - 2 \tilde{x}^T \tilde{y} + 1) \\
& = 1 - \mbox{SimilitudCoseno}(x,y).
\end{split}
\end{equation}

Podemos calcular la similitud del coseno entre los vectores TFIDF en nuestro corpus de la siguiente manera.

In [ ]:
import numpy as np

# Normalizamos los documentos
X_tfidf_norm = dense / np.linalg.norm(dense, axis=1)[:,None]
# Calculamos la similitud del coseno a partir de la última fórmula.
M = X_tfidf_norm @ X_tfidf_norm.T
print(M)